In [7]:
from pathlib import Path
import pandas as pd
import hvplot.pandas
import sqlite3 as sql3
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(Path('Resources/credit_card_data.csv'))
df.head()

,ID,MONTHS_BALANCE,STATUS,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008806,0.0,C,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,Security staff,2
1,5008808,0.0,0,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,Sales staff,1
2,5008810,0.0,C,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,Sales staff,1
3,5008811,0.0,C,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,Sales staff,1
4,5008815,0.0,0,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,-16872,-769,Accountants,2


In [3]:
df.nunique()

ID                     17043
MONTHS_BALANCE             1
STATUS                     8
CODE_GENDER                2
FLAG_OWN_CAR               2
FLAG_OWN_REALTY            2
CNT_CHILDREN               8
AMT_INCOME_TOTAL         189
NAME_INCOME_TYPE           5
NAME_EDUCATION_TYPE        5
NAME_FAMILY_STATUS         5
NAME_HOUSING_TYPE          6
DAYS_BIRTH              4792
DAYS_EMPLOYED           3124
OCCUPATION_TYPE           18
CNT_FAM_MEMBERS            9
dtype: int64

In [4]:
status_list = ['1', '2', '3', '4', '5']
drop_list = [
    'CODE_GENDER_1',
    'CODE_GENDER_F',
    'FLAG_OWN_CAR_1',
    'FLAG_OWN_CAR_N',
    'FLAG_OWN_REALTY_1',
    'FLAG_OWN_REALTY_N',
    'NAME_INCOME_TYPE_1',
    'NAME_EDUCATION_TYPE_1',
    'NAME_FAMILY_STATUS_1',
    'NAME_HOUSING_TYPE_1',
    'OCCUPATION_TYPE_1'
    ]

In [5]:
df_status_binned = df.drop(columns='ID')
# The following code will bin all numbers +=1 in the STATUS column, however it will create extra collumns
# The extra columns are deleted using the drop list above
df_status_binned = df_status_binned.where(~df_status_binned['STATUS'].isin(status_list), 1)
df_status_binned.nunique()
decoded_df = pd.get_dummies(df_status_binned, dtype=int)
decoded_df = decoded_df.drop(columns=drop_list)
decoded_df.head()

,MONTHS_BALANCE,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,STATUS_1,STATUS_0,STATUS_C,STATUS_X,...,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff
0,0.0,0,112500.0,-21474,-1134,2,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0.0,0,270000.0,-19110,-3051,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0.0,0,270000.0,-19110,-3051,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0.0,0,270000.0,-19110,-3051,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,0.0,0,270000.0,-16872,-769,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Scaling

This section will examine some of the different preprocessing scalers from SKlearn found in the doccumentation here: `https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#plot-all-scaling-standard-scaler-section`

StandardScaler:
* \+ Will shrink the `AMT_INCOME_TOTAL`, `DAYS_BIRTH`, and `DAYS_EMPLOYED` columns to fit into a range based off the variance
* \+ All values which are binary (either 1 or 0) should remain the same as there is only 1 value of unit variance
* \- The range may be difficult to understand for exploratory purposes

MaxAbsScaler:
* \+ A MinMax Scaler which places values between \[-1,1\]
* \+ The range is easier to understand as every feature is on the same scale
* \- As each feature is on the same scale, values between 1 and zero will probably be very small values who's meaning may get lost

RobustScaler:
* \+ The scale is based off percentiles rather than variance and will therefore not be influenced by outliers
* \+ Basically just the standard scaler, but with a range that is easier to understand

In [13]:
s_scaler = StandardScaler()
ma_scaler = MaxAbsScaler()
r_scaler = RobustScaler()
df_normalised_ss = s_scaler.fit_transform(decoded_df)
df_normalised_mas = ma_scaler.fit_transform(decoded_df)
df_normalised_rs = r_scaler.fit_transform(decoded_df)

[[-0.11747548 -0.64004915 -0.75246761 ... -0.07604878  6.50048807
  -0.07121553]
 [-0.11747548 -0.64004915  0.72724283 ... -0.07604878 -0.1538346
  -0.07121553]
 [-0.11747548 -0.64004915  0.72724283 ... -0.07604878 -0.1538346
  -0.07121553]
 ...
 [-0.11747548 -0.64004915 -0.3296932  ... -0.07604878 -0.1538346
  -0.07121553]
 [ 8.51241487  0.63697704 -1.80939424 ... -0.07604878 -0.1538346
  -0.07121553]
 [-0.11747548 -0.64004915 -0.75246761 ... -0.07604878 -0.1538346
  -0.07121553]]


### Standard Scaler PCA and KMeans clustering

In [19]:
pca = PCA(n_components=2)
loan_pca = pca.fit_transform(df_normalised_ss)
var_ratio = pca.explained_variance_ratio_
print(loan_pca[:5], var_ratio)

[[-1.64963276 -1.09690795]
 [ 0.46601216 -2.34308799]
 [ 0.36909321 -2.3620186 ]
 [ 0.36909321 -2.3620186 ]
 [-0.89211084  1.4393758 ]] [0.05799637 0.05053174]


In [15]:
loan_pca_df = pd.DataFrame(
    loan_pca,
    columns=['PCA1', 'PCA2']
)

inertia = []
k = list(range(1,11))

for j in k:
    k_model = KMeans(n_clusters=j, random_state=129)
    k_model.fit(loan_pca_df)
    inertia.append(k_model.inertia_)

pca_elbow_data = {'k': k, 'inertia' : inertia}
pd.DataFrame(pca_elbow_data).hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xticks=k
)

:Curve   [k]   (inertia)

In [26]:
# Calculating variance ratio of first two PCA variables
print(f"The Variance ratio of the first two PCA variables is: {round((0.05053174 + 0.05799637) * 100)}%")

The Variance ratio of the first two PCA variables is: 11%


The variance ratio for two PCA variables is 0.05799637 and 0.05053174 which is only 11% of the total variance. This is not good at all.

3 clusters seems to be an appropriate number of clusters for Kmeans clustering.

In [16]:
model = KMeans(n_clusters=3, random_state=129)
model.fit(loan_pca_df)
k_3 = model.predict(loan_pca_df)

loan_pca_predictions_df = loan_pca_df.copy()
loan_pca_predictions_df['loan_segments'] = k_3

In [17]:
loan_pca_predictions_df.hvplot.scatter(
    x='PCA1',
    y='PCA2',
    by='loan_segments'
)

:NdOverlay   [loan_segments]
   :Scatter   [PCA1]   (PCA2)

The scattering is showing large clumping with most loan segments 1 PCA2 value being below 1 and loan segments 0 PCA2 value being mostly above 0. There is also a huge clump with all loan segments 2 having the exact same PCA1 and PCA2 values. Zooming in does not show any sub-clustering.

This is not a good scattering.

### MaxAbs Scaler PCA and KMeans clustering

In [23]:
pca = PCA(n_components=2)
loan_pca = pca.fit_transform(df_normalised_mas)
var_ratio = pca.explained_variance_ratio_
print(loan_pca[:5], var_ratio)

[[-0.74236148 -0.89416703]
 [ 0.40838191  1.48787217]
 [ 0.37064941  0.49419082]
 [ 0.37064941  0.49419082]
 [ 0.37114859 -0.30589767]] [0.12119654 0.09805741]


In [24]:
loan_pca_df = pd.DataFrame(
    loan_pca,
    columns=['PCA1', 'PCA2']
)

inertia = []
k = list(range(1,11))

for j in k:
    k_model = KMeans(n_clusters=j, random_state=129)
    k_model.fit(loan_pca_df)
    inertia.append(k_model.inertia_)

pca_elbow_data = {'k': k, 'inertia' : inertia}
pd.DataFrame(pca_elbow_data).hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xticks=k
)

:Curve   [k]   (inertia)

In [27]:
print(f"The variance ratio of the first two PCA variables is: {round((0.12119654 + 0.09805741) * 100)}%")

The variance ratio of the first two PCA variables is: 22%


The variance ratio for this scaling is a bit better at 22%. It is still not ideal.

For the ideal number of clusters, I think that either 3 or 5 would work well for this.

In [28]:
model = KMeans(n_clusters=3, random_state=129)
model.fit(loan_pca_df)
k_3 = model.predict(loan_pca_df)

loan_pca_predictions_df = loan_pca_df.copy()
loan_pca_predictions_df['loan_segments'] = k_3

In [29]:
loan_pca_predictions_df.hvplot.scatter(
    x='PCA1',
    y='PCA2',
    by='loan_segments'
)

:NdOverlay   [loan_segments]
   :Scatter   [PCA1]   (PCA2)

In [30]:
loan_pca_df = pd.DataFrame(
    loan_pca,
    columns=['PCA1', 'PCA2']
)

model = KMeans(n_clusters=5, random_state=129)
model.fit(loan_pca_df)
k_5 = model.predict(loan_pca_df)

loan_pca_predictions_df = loan_pca_df.copy()
loan_pca_predictions_df['loan_segments'] = k_5

In [31]:
loan_pca_predictions_df.hvplot.scatter(
    x='PCA1',
    y='PCA2',
    by='loan_segments'
)

:NdOverlay   [loan_segments]
   :Scatter   [PCA1]   (PCA2)

This clustering looks a lot better than the previous one, as there is no outliers which skew the graph.

I will have to check the cluster numbers against the origninal dataset to prove any correlations.

### Robust Scaler PCA and KMeans clustering

In [32]:
pca = PCA(n_components=2)
loan_pca = pca.fit_transform(df_normalised_rs)
var_ratio = pca.explained_variance_ratio_
print(loan_pca[:5], var_ratio)

[[-0.00603905 -0.85367753]
 [-1.82104171 -0.28354796]
 [-1.82592263 -0.2923834 ]
 [-1.82592263 -0.2923834 ]
 [-0.86069855  0.67159108]] [0.18663135 0.18192801]


In [33]:
loan_pca_df = pd.DataFrame(
    loan_pca,
    columns=['PCA1', 'PCA2']
)

inertia = []
k = list(range(1,11))

for j in k:
    k_model = KMeans(n_clusters=j, random_state=129)
    k_model.fit(loan_pca_df)
    inertia.append(k_model.inertia_)

pca_elbow_data = {'k': k, 'inertia' : inertia}
pd.DataFrame(pca_elbow_data).hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xticks=k
)

:Curve   [k]   (inertia)

In [34]:
print(f"The variance ratio of the first two PCA variables is: {round((0.18663135 + 0.18192801) * 100)}%")

The variance ratio of the first two PCA variables is: 37%


The robust scaling seems to have the highest variance ratio in the first two PCA variables of the lot at 37%. This is still not ideal.

Again, 3 seems to be the optimal number for KMeans clusters.

In [35]:
model = KMeans(n_clusters=3, random_state=129)
model.fit(loan_pca_df)
k_3 = model.predict(loan_pca_df)

loan_pca_predictions_df = loan_pca_df.copy()
loan_pca_predictions_df['loan_segments'] = k_3

In [36]:
loan_pca_predictions_df.hvplot.scatter(
    x='PCA1',
    y='PCA2',
    by='loan_segments'
)

:NdOverlay   [loan_segments]
   :Scatter   [PCA1]   (PCA2)

This one seems like a good form of clustering. There are outliers visible, but there are no large clumps of data with the same PCA values overlapping eachother. Zooming into the main cluster, we can see diagonal lines differentiating groups of sub-clusters.

I think this is a good form of clustering which could be explored further.